# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [3]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-4o",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [4]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the 2010 Formula One World Championship. He drove for the Red Bull Racing team and secured his first world title that year, becoming the youngest World Champion in F1 history at that time.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [17]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.  
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [18]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.  
Team: Renault.


In [19]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.  
Team: Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [20]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.  
Team: Mercedes.  
Points: 413.  


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [21]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Drive: Lewis Hamilton  
Team: Mercedes  
Points: 413  


In [22]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.  
Team: Renault F1 Team.


Few Shots for classification.


In [23]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Negative


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

## Version 1 - Finals MVP in NBA championship

In [45]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in NBA.'}
]
print(return_OAIResponse("Who was the MVP of the finals in the NBA championship in 2016?", context_user))

The MVP of the NBA Finals in 2016 was LeBron James. He won the award after leading the Cleveland Cavaliers to a historic comeback victory against the Golden State Warriors, overcoming a 3-1 series deficit to win the championship.


In [49]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in NBA.

     Who was the MVP of the finals in the NBA championship in 2015?
     Finals MVP: Andre Iguodala.
     Team: Golden State Warriors."""}
]
print(return_OAIResponse("Who was the MVP of the finals in the NBA championship in 2016?", context_user))

The MVP of the NBA Finals in 2016 was LeBron James. He led the Cleveland Cavaliers to victory over the Golden State Warriors.


In [48]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in NBA.

     Who was the MVP of the finals in the NBA championship in 2015?
     Finals MVP: Andre Iguodala.
     Team: Golden State Warriors.

     Who was the MVP of the finals in the NBA championship in 2016?
     Finals MVP: LeBron James.
     Team: Cleveland Cavaliers."""}
]
print(return_OAIResponse("Who was the MVP of the finals in the NBA championship in 2017?", context_user))

Finals MVP: Kevin Durant.  
Team: Golden State Warriors.


## Version 2 - Most populated capital

In [50]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in demography accross the world.'}
]
print(return_OAIResponse("What is the most populated capital in Europe?", context_user))

As of the latest data, the most populated capital in Europe is Moscow, Russia. Moscow has a population exceeding 12 million people, making it not only the largest capital in Europe in terms of population but also one of the largest cities on the continent. Keep in mind that population figures can change over time due to various factors, including urbanization, migration, and administrative changes.


In [59]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in demography accross the world.

     Pattern to follow:
     What is the most populated capital in Europe?"
     Capital : Moscow.
     Country: Russia.
     Population: 13,258,262."""}
]
print(return_OAIResponse("What is the most populated capital in Africa?", context_user))

Capital: Cairo.  
Country: Egypt.  
Population: 21,750,020.


In [60]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in demography accross the world.

     Pattern to follow:
     What is the most populated capital in Europe?
     Capital : Moscow.
     Country: Russia.
     Population: 13,258,262

     What is the most populated capital in Africa?
     Capital : Kinshasa.
     Country: Democratic Republic of the Congo .
     Population: over 15 million."""}
]
print(return_OAIResponse("What is the most populated capital in Asia?", context_user))

Capital: Beijing.  
Country: China.  
Population: over 21 million.


## Version 3 - Strongest character in the MCU

In [ ]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in the MCU.'}
]
print(return_OAIResponse("What is the strongest characher in the MCU?", context_user))

Determining the "strongest" character in the Marvel Cinematic Universe (MCU) can be subjective and depends on the criteria used, such as physical strength, magical power, durability, or overall influence.

1. **Physical Strength**: Characters like the Hulk and Thor are often at the top when considering sheer physical power. Both have showcased immense strength in battles.

2. **Magical Power**: Characters like Doctor Strange and Scarlet Witch are incredibly powerful in terms of magical abilities. Scarlet Witch, in particular, has shown the capability to alter reality and take on entire groups of heroes with her abilities.

3. **Overall Power**: In terms of combined strength, durability, intelligence, and other powers, Captain Marvel is often considered one of the most powerful characters. She's shown to possess incredible energy, strength, and the ability to withstand significant damage.

4. **Celestial Beings**: Characters like Ego the Living Planet and the Celestials have powers that

In [64]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in the MCU.

     Pattern to follow:
     What is the strongest characher in the MCU?"
     Name : Hulk.
     Main power: Physical Strength.
     Origin: Earth."""}
]
print(return_OAIResponse("What is the most seen characher in the MCU?", context_user))

Name: Tony Stark/Iron Man.  
Main appearances: Tony Stark, portrayed by Robert Downey Jr., has appeared in multiple MCU films including all the "Iron Man" and "Avengers" films, as well as "Captain America: Civil War," "Spider-Man: Homecoming," and more.  
Origin: Earth.


In [ ]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in the MCU.

     Pattern to follow:
     What is the strongest characher in the MCU?"
     Name: Hulk.
     Main power: Physical Strength
     Main appearance:
        9 movies :\n
        All the "Avengers" films :\n
        - The Avengers (2012)\n
        - Avengers: Age of Ultron (2015)\n
        - Avengers: Infinity War (2018)\n
        - Avengers: Endgame (2019)\n
        The Incredible Hulk (2008)\n
        Thor: Ragnarok (2017)\n
     Origin: Earth.

     What is the most seen characher in the MCU?
     Name: Tony Stark/Iron Man.
     Main power: Powered armored suit
     Main appearances:
        9 movies :\n
        All the "Iron Man" films :\n
        - Iron Man (2008)\n
        - Iron Man 2 (2010)\n
        - Iron Man 3 (2013)\n
        All the "Avengers" films :\n
        - The Avengers (2012)\n
        - Avengers: Age of Ultron (2015)\n
        - Avengers: Infinity War (2018)\n
        - Avengers: Endgame (2019)\n
        Captain America: Civil War (2016)\n
        Spider-Man: Homecoming (2017)\n
     Origin: Earth."""}
]
print(return_OAIResponse("What is the youngest characher in the MCU?", context_user))

Name: Peter Parker/Spider-Man.
Main power: Superhuman strength, agility, and the ability to cling to most surfaces; spider-sense; web-shooting.
Main appearances:
   7 movies:

   - Captain America: Civil War (2016)

   - Spider-Man: Homecoming (2017)

   - Avengers: Infinity War (2018)

   - Avengers: Endgame (2019)

   - Spider-Man: Far From Home (2019)

   - Spider-Man: No Way Home (2021)

   - Spider-Man: Freshman Year (upcoming animated series)

Origin: Earth.


In [67]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in the MCU.

     Pattern to follow:
     What is the strongest characher in the MCU?"
     Name: Hulk.
     Main power: Physical Strength
     Main appearance:
        9 movies :\n
        All the "Avengers" films :\n
        - The Avengers (2012)\n
        - Avengers: Age of Ultron (2015)\n
        - Avengers: Infinity War (2018)\n
        - Avengers: Endgame (2019)\n
        The Incredible Hulk (2008)\n
        Thor: Ragnarok (2017)\n
     Origin: Earth.

     What is the most seen characher in the MCU?
     Name: Tony Stark/Iron Man.
     Main power: Powered armored suit
     Main appearances:
        9 movies :\n
        All the "Iron Man" films :\n
        - Iron Man (2008)\n
        - Iron Man 2 (2010)\n
        - Iron Man 3 (2013)\n
        All the "Avengers" films :\n
        - The Avengers (2012)\n
        - Avengers: Age of Ultron (2015)\n
        - Avengers: Infinity War (2018)\n
        - Avengers: Endgame (2019)\n
        Captain America: Civil War (2016)\n
        Spider-Man: Homecoming (2017)\n
     Origin: Earth."""}
]
print(return_OAIResponse("What is the youngest characher in the MCU but not from earth?", context_user))

Name: Groot.
Main power: Regeneration and control over plant life.
Main appearances:
   5 movies:

   Guardians of the Galaxy (2014)

   Guardians of the Galaxy Vol. 2 (2017)

   Avengers: Infinity War (2018)

   Avengers: Endgame (2019)

   Thor: Love and Thunder (2022) 

Origin: Planet X.


## My observations:
I used GPT-4o exclusively and experimented with prompt variations.
I noticed that, at times, the model requires explicit encouragement to follow the provided pattern, even when it seems clear.